<a href="http://landlab.github.io"><img style="float: left" src="../../landlab_header.png"></a>

# Using the any-grid-type Landlab DepressionFinderAndRouter and FlowRouter to solve depressions and direct and accumulate flow.


In this tutorial we will present:
* the way the DepressionFinderAndRouter handles depressions in flow routing,
* the way the FlowRouter does the same thing,
* the compatibility of the FlowRouter with other flow directors, inc. FlowDirectorMFD,
* and run a simple landscape evolution model that uses the FlowRouter.

The FlowRouter calculates single-to-single flow directions and accumulations, and handles depression at the same time. Compared with the DepressionFinderAndRouter, the component works on NetworkModelGrids, is compatible with multiple flow, and uses a fast implementation of the priority flood flow router algorithm. That being said, due to the algorithm, the FlowRouter might underperform the DepressionFinderAndRouter for grids without depressions.

NB: This notebook has not been validated on windows platforms.

A complementary component, the PriorityFloodFlowRouter, also handles depressions in RasterModelGrids only and is not compared here (see  [the_Flow_Director_Accumulator_PriorityFlood notebook](the_Flow_Director_Accumulator_PriorityFlood.ipynb)).

*(Priority Flood Flow algorithm and downstream to upstream node ordering adapted from Barnes et al., 2014 and Braun and Willett, 2013).*

## 1. Grid setting.
Here, we will create a new grid of any type, define its base-level and its topography.

### 1.1. Grid creation.
First we will create a grid of a specific type. Let's import the required python and Landlab libraries.

In [ ]:
# Libraries required
####################
# Python libraries
# import time  # performance testing

import sys as sys

import matplotlib.collections as mpl_collections
import matplotlib.pyplot as plt
import numpy as np

flow_router_notebook_platforms = ["linux", "cygwin", "darwin"]
# There is a problem on windows platform on github with 'win32' 2024-04-22

# import landlab plotting functionality
# Landlab libraries/methods
# Grid management
from landlab import (
    FramedVoronoiGrid,
    HexModelGrid,
    NetworkModelGrid,
    RadialModelGrid,
    RasterModelGrid,
    VoronoiDelaunayGrid,
    imshow_grid,
)

# Depression and flow handling
from landlab.components import DepressionFinderAndRouter, FlowAccumulator, FlowRouter
from landlab.plot import graph
from landlab.plot.drainage_plot import drainage_plot

We then setup the type, the size and the spacing of the grid. Let's choose a hexagonal grid and display the nodes. 

*Note that you can select another type of grid, e.g. VoronoiDelaunayGrid by changing* ```[0]``` *into* ```[4]``` *in the 2nd line of the code below.*

In [ ]:
# Selection of the grid type
model_grid = (
    RasterModelGrid,
    HexModelGrid,
    RadialModelGrid,
    FramedVoronoiGrid,
    VoronoiDelaunayGrid,
    NetworkModelGrid,
)[0]

# Grid parameters
xy_spacing = (10, 10)
# Random generator to generate the coordinates of a VoronoiDelaunayGrid. To changeµ the coordinates,
# change the seed (but you'll have to change the coordinates of perimeter and base-level nodes in next cell)
random_generator = np.random.Generator(np.random.PCG64(seed=200))
grid_param = {
    RasterModelGrid: {
        "shape": (10, 10),
        "xy_spacing": xy_spacing,
        "xy_axis_units": "m",
    },
    HexModelGrid: {
        "shape": (9, 5),
        "spacing": xy_spacing[0],
        "node_layout": ["hex", "rect"][0],
        "xy_axis_units": "m",
    },
    RadialModelGrid: {
        "n_rings": 8,
        "nodes_in_first_ring": 4,
        "spacing": xy_spacing[0],
        "xy_axis_units": "m",
    },
    FramedVoronoiGrid: {
        "shape": (7, 7),
        "xy_spacing": (1.0, 1.0),
        "xy_min_spacing": (0.25, 0.25),
        "seed": 200,
        "xy_axis_units": "m",
    },
    VoronoiDelaunayGrid: {
        "x": 10 * xy_spacing[0] * random_generator.random(50),  # 00),
        "y": 10 * xy_spacing[0] * random_generator.random(50),  # 00),
        "xy_axis_units": "m",
    },
    NetworkModelGrid: {
        "yx_of_node": (
            (0, 100, 200, 200, 300, 400, 400, 125),
            (0, 0, 100, -50, -100, 50, -150, -100),
        ),
        "links": ((1, 0), (2, 1), (1, 7), (3, 1), (3, 4), (4, 5), (4, 6)),
        "xy_axis_units": "m",
    },
}[model_grid]

# Creation of the grid
g = model_grid(**grid_param)
nodes_n = g.number_of_nodes
# Create field topographic elevation at nodes with value 0 (will be modified later)
z = g.add_zeros("topographic__elevation", at="node")

if nodes_n < 200:
    graph.plot_graph(g, at="node")

### 1.2. Base-level and outlets.
We now need to define the base-level of the grid.

Landlab grids consist of nodes (and other elements as cells, links, ...) which have 4 status. (1) Nodes that receives flow from other nodes and transfer flow to other nodes are **core** nodes. (2) and (3), Nodes that receive flow but don't transfer it are either **fixed value** or **fixed gradient**. These nodes form the base-level of the grid and can be outlets of the grid (and they can have different elevation depending on the location on the grid. (4) Nodes that neither receive nor transfer flow are **closed**.

By default, all grids except VoronoiDelaunayGrid and NetworkModelGrid have their base-level defined at their **perimeter**, which is also called **boundary**. In other words, the perimeter nodes have the status fixed value. In the VoronoiDelaunayGrid, the perimeter nodes are also of fixed values but are not always exhaustively determined. This problem is linked to the method to catch the perimeter, which is not appropriate to all node distributions. The perimeter nodes of the NetworkModelGrid are by default all with status=core. So, if we choose VoronoiDelaunayGrid and NetworkModelGrid, it can be appropriate to specifically tell the code where the base-level nodes are, using the status fixed value and closed. Presently, the method to catch the perimeter nodes doesn't work on all VoronoiDelaunayGrids depending on the distribution of the nodes and for completely random VoronoiDelaunay grids, base-level nodes need to be redefined.

Here, we study two cases: either there's only one base-level outlet, at the node id 0, or all perimeter nodes are base-level except on one edge (*Note that this latter case is not realistic for NetworkModelGrids*). We start by the second case.

*You can investigate the first case by replacing* ```[1]``` *by* ```[0]``` *in the 2nd line of the code below.*


In [ ]:
base_level_selection = [
    "one perimeter node only (the node with id 0) is base-level",
    "all perimeter nodes are base-level except for one perimeter edge",
][1]

if base_level_selection == "one perimeter node only (the node with id 0) is base-level":
    if isinstance(g, RasterModelGrid):
        # with methods
        g.set_closed_boundaries_at_grid_edges(
            right_is_closed=True,
            top_is_closed=True,
            left_is_closed=True,
            bottom_is_closed=True,
        )
        g.status_at_node[0] = g.BC_NODE_IS_FIXED_VALUE
    elif isinstance(g, HexModelGrid):
        g.status_at_node[g.perimeter_nodes] = g.BC_NODE_IS_CLOSED
        g.status_at_node[0] = g.BC_NODE_IS_FIXED_VALUE
    elif isinstance(g, RadialModelGrid):
        g.status_at_node[g.perimeter_nodes] = g.BC_NODE_IS_CLOSED
        g.status_at_node[0] = g.BC_NODE_IS_FIXED_VALUE
    # manually for VoronoiDelaunayGrid. Note that the node ids are for a random distribution of nodes with seed = 200
    elif isinstance(g, VoronoiDelaunayGrid):
        g.status_at_node[
            [4, 8, 25, 31, 44, 49]
            + [40, 42, 46, 48, 47, 45]
            + [37, 18, 9, 5, 0]
            + [1, 2, 3, 7]
        ] = g.BC_NODE_IS_CLOSED
    elif isinstance(g, NetworkModelGrid):
        g.status_at_node[0] = g.BC_NODE_IS_FIXED_VALUE

elif (
    base_level_selection
    == "all perimeter nodes are base-level except for one perimeter edge"
):
    # with existing methods
    if isinstance(g, RasterModelGrid):
        g.set_closed_boundaries_at_grid_edges(
            right_is_closed=False,
            top_is_closed=True,
            left_is_closed=False,
            bottom_is_closed=False,
        )
    elif isinstance(g, HexModelGrid):
        g.status_at_node[g.nodes_at_top_edge] = g.BC_NODE_IS_CLOSED
    # manually
    elif isinstance(g, RadialModelGrid):
        g.status_at_node[[0, 1, 3, 8, 15, 24, 35, 49, 64]] = g.BC_NODE_IS_CLOSED
    elif isinstance(g, VoronoiDelaunayGrid):
        g.status_at_node[[4, 6, 8, 25, 31, 44, 49]] = g.BC_NODE_IS_CLOSED

    # NB: this case is not really realistic with a NetworkModelGrid, which represents a river network rather than a topography.

The base-level and closed nodes will be visualized on the grid by a specific display method ```drainage_plot()``` after running the flow directions. For RasterModelGrids only, the closed perimeter nodes can also be visualized by the black color using the method imshow_grid (see below).

### 1.3. Topography.
We now generate a random topography on which we'll direct and accumulate the flow. We fix topography of the node of id 0 at zero. And we create a wide depression in the grid. 

Note that it's also possible to direct and accumulate flow on other surface than the topographic elevation.

For NetworkModelGrids, we use a specific way to display the topography since ```imshow_grid()``` is currently not implemented for NetworkModelGrids.

*To generate a different topography, you can modify* ```seed=500``` *by* ```seed=200``` *for instance, or remove the seed.*

In [ ]:
# set constant random seed for "stable" random pull
random_generator = np.random.Generator(np.random.PCG64(seed=500))
random_topography = 10.0  # range in elevation (from 0)
z = g.at_node["topographic__elevation"] = (
    random_topography * random_generator.random(nodes_n) + 1.0
)
z[0] = 0.0

# Add a large depression within the grid
depression_nodes = []
if isinstance(g, RasterModelGrid):
    depression_nodes = [43, 44, 45, 53, 54, 55, 63, 64, 65, 66, 74, 75]
elif isinstance(g, HexModelGrid):
    depression_nodes = [29, 30, 37, 38, 39, 40, 45, 46, 47]
elif isinstance(g, RadialModelGrid):
    depression_nodes = [12, 13, 14, 21, 22, 23, 24, 30, 32, 33, 46]
elif isinstance(g, FramedVoronoiGrid):
    depression_nodes = [17, 18, 23, 24, 25, 32]
elif isinstance(g, VoronoiDelaunayGrid):
    depression_nodes = [14, 16, 22, 23, 24, 29, 33]
z[depression_nodes] = random_generator.random(len(depression_nodes))

# Display
if not isinstance(g, NetworkModelGrid):
    imshow_grid(
        g,
        "topographic__elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Topographic elevation",
        show_elements=True,
    )
else:
    # imshow_grid is not available for NetworkModelGrid, we use the classic matplotlib
    plt.title("Topographic elevation")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")

    link_coordinates = []
    for nodes in g.nodes_at_link:
        link_coordinates.append(
            [
                (g.x_of_node[nodes[0]], g.y_of_node[nodes[0]]),
                (g.x_of_node[nodes[1]], g.y_of_node[nodes[1]]),
            ]
        )
    lc = mpl_collections.LineCollection(link_coordinates)
    plt.gca().add_collection(lc)
    plt.scatter(
        g.x_of_node, g.y_of_node, s=200, c=z, cmap="terrain", edgecolors="black"
    )
    plt.colorbar(label="Elevation (m)")
    plt.autoscale()


plt.show()

For RasterModelGrids: in this display, you can see the perimeter nodes for the RasterModelGrid. The closed perimeter nodes are black-colored. <br>
For other types of grids: the perimeter nodes are hidden because of the implementation of the method ```imshow_grid()```. But for VoronoiDelaunayGrid, a few perimeter nodes can still be visible due to the incorrect determination of the perimeter nodes for this type of grid.

Note the blue-tainted depression.

## 2. The DepressionFinderAndRouter.

Here we will run flow directions and accumulations (water discharge) and correct them by the DepressionFinderAndRouter which handles depressions. Presently, this approach works with all grids **except the NetworkModelGrid**. Additionally, this approach presently only works with single-to-single flow.

### 2.1. Flow directions and accumulation runs with sinks.
First, we calculate flow directions and accumulations, without dealing with sinks (term equivalent to pit in this notebook). Calculations are done with a single-to-single flow, which means that we suppose that all the flow is directed from a donor node to only one receiver node. This is done by using the steepest slope from a donor node.

In [ ]:
# We take into account diagonal links for RasterModelGrids but not for other grids
flow_director = "D8" if isinstance(g, RasterModelGrid) else "D4"
if not isinstance(g, NetworkModelGrid):
    accumulator = FlowAccumulator(g, flow_director=flow_director)
    accumulator.run_one_step()
    drainage_plot(g, surf_cmap="terrain")
    plt.show()

The drainage is displayed by the ```drainage_plot()``` method (which is not implemented for NetworkModelGrid). The cells are colored according to their elevation (right colorbar), except for perimeter nodes of non RasterModelGrids. The flow is directed along the steepest slope (yellow arrows). The final outlet of the catchments is either a base-level perimeter node or a core node which reveals to be a sink.

In [ ]:
if not isinstance(g, NetworkModelGrid):
    imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )
else:
    # imshow_grid is not available for NetworkModelGrid, we use the classic matplotlib
    plt.title("Surface water discharge")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")

    link_coordinates = []
    for nodes in g.nodes_at_link:
        link_coordinates.append(
            [
                (g.x_of_node[nodes[0]], g.y_of_node[nodes[0]]),
                (g.x_of_node[nodes[1]], g.y_of_node[nodes[1]]),
            ]
        )
    lc = mpl_collections.LineCollection(link_coordinates)
    plt.gca().add_collection(lc)
    plt.scatter(
        g.x_of_node,
        g.y_of_node,
        s=200,
        c=g.at_node["surface_water__discharge"],
        cmap="Blues",
        edgecolors="black",
    )
    plt.colorbar(label="Elevation (m)")
    plt.autoscale()

Water discharge reflects the presence of sinks within the core nodes.

### 2.2. DepressionFinderAndRouter run.

While sinks can occur naturally, they can also be produced by DEM artefacts linked to their resolution. Additionally, on timescales larger than 1,000 years, depressions (with their sink as their lowest point) tend to be breached, but the erosion laws currently used might have difficulty to simulate this breaching. This gives ground to prefer to direct the flow outside of the depression, to ensure that all outlets of the grid are on the perimeter.

This is done here by the DepressionFinderAndRouter. This component scans all the sinks. These sinks are provided by the FlowDirector/Accumulator (field ```flow__sink_flag```). The component then maps depressions and finds the lowest node at the perimeter of the depression. Once found, it considers this point as the depression outlet and recalculates all flow directions and accumulations by evacuating the depression.

In [ ]:
if not isinstance(g, NetworkModelGrid):
    finder = DepressionFinderAndRouter(g, routing=flow_director)
    finder.map_depressions()
    drainage_plot(g, surf_cmap="terrain")
    plt.show()
    imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )
else:
    # imshow_grid is not available for NetworkModelGrid, we use the classic matplotlib
    plt.title("Surface water discharge")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")

    link_coordinates = []
    for nodes in g.nodes_at_link:
        link_coordinates.append(
            [
                (g.x_of_node[nodes[0]], g.y_of_node[nodes[0]]),
                (g.x_of_node[nodes[1]], g.y_of_node[nodes[1]]),
            ]
        )
    lc = mpl_collections.LineCollection(link_coordinates)
    plt.gca().add_collection(lc)
    plt.scatter(
        g.x_of_node,
        g.y_of_node,
        s=200,
        c=g.at_node["surface_water__discharge"],
        cmap="Blues",
        edgecolors="black",
    )
    plt.colorbar(label="Elevation (m)")
    plt.autoscale()

All depressions were handled and the flow is now directed to base-level perimeter outlet nodes. Flow accumulation and water discharge is also updated.

## 3. The FlowRouter.

Now we will see how to handle depressions with the FlowRouter, compatible with all types of grids and with single-to-single and single-to-multiple flow.

### 3.1. FlowRouter runs with single-flow.

Let's start by running the FlowRouter on the grid.

In [ ]:
router = FlowRouter(g)
if sys.platform in flow_router_notebook_platforms:
    router.run_one_step()

if not isinstance(g, NetworkModelGrid):
    drainage_plot(g, surf_cmap="terrain")
    plt.show()
    imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )
else:
    # imshow_grid is not available for NetworkModelGrid, we use the classic matplotlib
    plt.title("Surface water discharge")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")

    link_coordinates = []
    for nodes in g.nodes_at_link:
        link_coordinates.append(
            [
                (g.x_of_node[nodes[0]], g.y_of_node[nodes[0]]),
                (g.x_of_node[nodes[1]], g.y_of_node[nodes[1]]),
            ]
        )
    lc = mpl_collections.LineCollection(link_coordinates)
    plt.gca().add_collection(lc)
    plt.scatter(
        g.x_of_node,
        g.y_of_node,
        s=200,
        c=g.at_node["surface_water__discharge"],
        cmap="Blues",
        edgecolors="black",
    )
    plt.colorbar(label="Elevation (m)")
    plt.autoscale()

The FlowRouter handles the depressions and calculates the single-to-single flow directions at the same time. For this, the component starts from the base-level nodes (with status fixed value or fixed gradient) and constructs a flow path by overcoming the depressions. It ensures that peaks remain peaks by always handling nodes with the lowest elevation first. By default for rasters, the component also diagonals (this option can be modified by the argument ```diagonals=False```).

The flows are directed differently from the DepressionFinderAndRouter because the algorithms are different. The FlowRouter algorithm starts from base-level nodes to construct the paths by simultaneously solving the depressions (this is the **priority flood flow algorithm**). Conversely, the FlowDirector calculates donors and receivers without starting from the base-level nodes and after that, the DepressionFinderAndRouter scans the depressions to solve them.

### 3.2. FlowRouter with multiple flows.
It's also possible to combine the FlowRouter with multiple flow directions and accumulations. Note that multiple flow is not implemented for NetworkModelGrids.

Let's delete the fields which need to be resized in the multiple flow configuration.

In [ ]:
# Delete fields used with different shapes in FlowDirectorMFD
for field in [
    "drainage_area",
    "flow__link_to_receiver_node",
    "flow__receiver_node",
    "flow__receiver_proportions",
    "flow__upstream_node_order",
    "outlet_node",
    "surface_water__discharge",
    "topographic__steepest_slope",
]:
    if field in g.at_node.keys():
        g.delete_field(loc="node", name=field)

Now, we can run the FlowRouter in multiple_flow mode with the argument ```single_flow=False```, for which it will only produce a depression free elevation and other informations as the depth of the depression, the outlet of the depression and the flooded status.

In [ ]:
# We take into account diagonal links for RasterModelGrids but not for other grids
router = FlowRouter(g, single_flow=False)
if sys.platform in flow_router_notebook_platforms:
    router.run_one_step()

if not isinstance(g, NetworkModelGrid):
    plt.show()
    imshow_grid(
        g,
        "depression_free__elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Depression free elevation",
        show_elements=True,
    )
else:
    # imshow_grid is not available for NetworkModelGrid, we use the classic matplotlib
    plt.title("Depression free elevation")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")

    link_coordinates = []
    for nodes in g.nodes_at_link:
        link_coordinates.append(
            [
                (g.x_of_node[nodes[0]], g.y_of_node[nodes[0]]),
                (g.x_of_node[nodes[1]], g.y_of_node[nodes[1]]),
            ]
        )
    lc = mpl_collections.LineCollection(link_coordinates)
    plt.gca().add_collection(lc)
    plt.scatter(
        g.x_of_node,
        g.y_of_node,
        s=200,
        c=g.at_node["depression_free__elevation"],
        cmap="terrain",
        edgecolors="black",
    )
    plt.colorbar(label="Elevation (m)")
    plt.autoscale()

The depression-free elevation is determined by starting from the elevation the outlet of the depression and going upstream. Depression-free elevation is calculated this way: depression-free elevation of the receiver + a small epsilon. This epsilon is there to create an artificial very low slope to direct the multiple flow.

This depression-free elevation surface is given as an input to the multiple flow directors.

Let's run the **FlowDirectorMFD**, a multiple flow director component implemented by adapting the Multiple Flow Direction (MFD) algorithm of [Quinn et al., 1991](https://doi.org/10.1002/hyp.3360050106).

In [ ]:
flow_director = "FlowDirectorMFD"
diagonals = True if isinstance(g, RasterModelGrid) else False
if not isinstance(g, NetworkModelGrid):
    accumulator = FlowAccumulator(
        g,
        surface="depression_free__elevation",
        flow_director=flow_director,
        diagonals=diagonals,
    )

    accumulator.run_one_step()
    drainage_plot(g, surface="depression_free__elevation", surf_cmap="terrain")
    plt.show()
    imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )

Now, let's run the FlowDirectorDinf, a dual-flow director component implemented by adapting the D-Infinity (DINF) algorithm of [Tarboton, 1997](https://doi.org/10.1029/96WR03137). Note that component only works for RasterGrids.

Again, we delete the fields which need to be resized in the multiple flow configuration and then we instantiate and run the component.

In [ ]:
# Delete fields used with different shapes in FlowDirectDINF
for field in [
    "drainage_area",
    "flow__link_to_receiver_node",
    "flow__receiver_node",
    "flow__receiver_proportions",
    "flow__upstream_node_order",
    "outlet_node",
    "surface_water__discharge",
    "topographic__steepest_slope",
]:
    if field in g.at_node.keys():
        g.delete_field(loc="node", name=field)

In [ ]:
flow_director = "FlowDirectorDINF"
if isinstance(g, RasterModelGrid):
    accumulator = FlowAccumulator(
        g, surface="depression_free__elevation", flow_director=flow_director
    )

    accumulator.run_one_step()
    drainage_plot(g, surface="depression_free__elevation", surf_cmap="terrain")
    imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )

### 3.3. The FlowRouter in a Landscape Evolution Model.

The FlowRouter may be convenient to run landscape evolution models (LEM) over large grids and a large number of iterations (because it's faster than the DrainageFinderAndRouter in this use case).

Let's start by deleting the fields which shape is for multiple flow directions.

In [ ]:
# Delete fields used with different shapes
for field in [
    "drainage_area",
    "flow__link_to_receiver_node",
    "flow__receiver_node",
    "flow__receiver_proportions",
    "flow__upstream_node_order",
    "outlet_node",
    "surface_water__discharge",
    "topographic__steepest_slope",
]:
    if field in g.at_node.keys():
        g.delete_field(loc="node", name=field)

Then, we construct a simple LEM. During 1 million years, the LEM will simulate an uplifting landscape from sea-level. Two erosion processes are included:
- bedrock mass wasting: slopes higher than a critical threshold are progressively chop down to the critical threshold (```ThresholdEroder```)
- river incision: rivers incise the bedrock with a stream power law (```StreamPowerEroder```).

We assume the system evacuates all sediment out of the grid.

In [ ]:
if sys.platform in flow_router_notebook_platforms:
    # Doesnt work yet with windows, for questions of int/int32/int64 compatibility with c/cc compiler on this platform.
    from tqdm import tqdm

    from landlab.components import SpaceLargeScaleEroder, ThresholdEroder

    # from landlab.components import StreamPowerEroder

    uplift_rate = 0.00001  # m/y
    dt = 1000  # timestep [y]
    t = 1e6  # 1e6 # totale time [y]
    n_dt = int(np.floor(t / dt))

    router = FlowRouter(g)
    landslider = ThresholdEroder(g)
    # eroder = StreamPowerEroder(g, K_sp=1e-6, m_sp=0.5, n_sp=1.0, erode_flooded_nodes=False)
    g.add_field("soil__depth", np.zeros(nodes_n), at="node", clobber=True)
    g.add_field("bedrock__elevation", z, at="node", clobber=True)
    eroder = SpaceLargeScaleEroder(g, K_br=8 * 1e-6, K_sed=1e-5)

    for i in tqdm(range(n_dt)):
        z[g.status_at_node == g.BC_NODE_IS_CORE] += uplift_rate * dt
        router.run_one_step()
        landslider.run_one_step()
        router.run_one_step()
        eroder.run_one_step(dt)

In [ ]:
if not isinstance(g, NetworkModelGrid):
    if nodes_n < 200:
        drainage_plot(g, surf_cmap="terrain")
    else:
        imshow_grid(
            g,
            "topographic__elevation",
            cmap="terrain",
            grid_units=("m", "m"),
            var_name="Elevation (m)",
            plot_name="Topographic elevation",
        )
    plt.show()
    imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
    )
else:
    # imshow_grid is not available for NetworkModelGrid, we use the classic matplotlib
    plt.title("Topographic elevation")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")

    link_coordinates = []
    for nodes in g.nodes_at_link:
        link_coordinates.append(
            [
                (g.x_of_node[nodes[0]], g.y_of_node[nodes[0]]),
                (g.x_of_node[nodes[1]], g.y_of_node[nodes[1]]),
            ]
        )
    lc = mpl_collections.LineCollection(link_coordinates)
    plt.gca().add_collection(lc)
    plt.scatter(
        g.x_of_node, g.y_of_node, s=200, c=z, cmap="terrain", edgecolors="black"
    )
    plt.colorbar(label="Elevation (m)")
    plt.autoscale()
    plt.show()

    plt.title("Surface water discharge")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")

    link_coordinates = []
    for nodes in g.nodes_at_link:
        link_coordinates.append(
            [
                (g.x_of_node[nodes[0]], g.y_of_node[nodes[0]]),
                (g.x_of_node[nodes[1]], g.y_of_node[nodes[1]]),
            ]
        )
    lc = mpl_collections.LineCollection(link_coordinates)
    plt.gca().add_collection(lc)
    plt.scatter(
        g.x_of_node,
        g.y_of_node,
        s=200,
        c=g.at_node["surface_water__discharge"],
        cmap="Blues",
        edgecolors="black",
    )
    plt.colorbar(label="Elevation (m)")
    plt.autoscale()

<br>
Author: Sebastien Lenard<br>
Date: 2022, Aug.

### Click here to learn about <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>